In [24]:
import os
import numpy as np
import sys
import time
import _init_paths
# from core.config import cfg, merge_cfg_from_file, merge_cfg_from_list, assert_and_infer_cfg
# from core.test_engine import run_inference, get_inference_dataset
# from datasets.json_dataset import JsonDataset
import pickle
import json
from utils.boxes import xywh_to_xyxy, bbox_overlaps
import ipdb

In [25]:
hico_pred_path = "/group/wanbo/HOI_plus/data/cache/hico_train_precomp_boxes_use_coco.json"
hico_gt_path = "/group/wanbo/HOI_plus/data/hico/hico_annotation.json"

hico_pred_path_test = "/group/wanbo/HOI_plus/data/cache/hico_train_precomp_boxes_use_coco.json"
hico_pred = json.load(open(hico_pred_path, 'r'))
hico_pred_test = json.load(open(hico_pred_path_test, 'r'))
hico_gt_path = json.load(open(hico_gt_path, 'r'))

In [26]:
# print(hico_pred.keys())
print(hico_pred['0'])

[[221.946044921875, 36.011962890625, 425.5889892578125, 326.634033203125, 0.9810082912445068, 1.0], [67.19271850585938, 92.02972412109375, 574.8057861328125, 399.3364562988281, 0.9993689656257629, 4.0]]


In [28]:
hico_gt_train = hico_gt_path['bbox_train']
hico_gt_train[0]
hico_gt_test = hico_gt_path['bbox_test']

print(hico_gt_train[0])
print(hico_gt_test[0])

x = np.ones((3,1))
x.repeat(1, 1)

{'bboxes': [[59, 98, 572, 405], [208, 33, 427, 300]], 'annotations': [{'obj_bbox_idx': 0, 'obj_cate_id': 4, 'action_cate_id': 50, 'action_cate_name': 'race', 'old_action_id': 152, 'obj_cate_name': 'motorcycle', 'hum_bbox_idx': 1}, {'obj_bbox_idx': 0, 'obj_cate_id': 4, 'action_cate_id': 45, 'action_cate_name': 'ride', 'old_action_id': 153, 'obj_cate_name': 'motorcycle', 'hum_bbox_idx': 1}, {'obj_bbox_idx': 0, 'obj_cate_id': 4, 'action_cate_id': 110, 'action_cate_name': 'sit_on', 'old_action_id': 154, 'obj_cate_name': 'motorcycle', 'hum_bbox_idx': 1}, {'obj_bbox_idx': 0, 'obj_cate_id': 4, 'action_cate_id': 100, 'action_cate_name': 'straddle', 'old_action_id': 155, 'obj_cate_name': 'motorcycle', 'hum_bbox_idx': 1}], 'wh': [480, 640], 'filename': 'HICO_train2015_00000001.jpg', 'run_id': 0, 'img_id': 1}
{'filename': 'HICO_test2015_00000001.jpg', 'annotations': [{'obj_cate_id': 14, 'obj_bbox_idx': 0, 'action_cate_id': 110, 'action_cate_name': 'sit_on', 'hum_bbox_idx': 1, 'obj_cate_name': 'be

array([[1.],
       [1.],
       [1.]])

In [31]:
total_person = 0
recall_person = 0

total_object = 0
recall_object = 0
print(len(hico_gt_train))
print(len(list(hico_pred.keys())))
for iddd, gt_info in enumerate(hico_gt_train):
    run_id = gt_info['run_id']
    bboxes = gt_info['bboxes']
    cates = [0] * len(bboxes)
    if len(gt_info['annotations'])!=0:
        for anno in gt_info['annotations']:
            cates[anno['obj_bbox_idx']] = anno['obj_cate_id']
            cates[anno['hum_bbox_idx']] = 1
    
        ## N1*5, N2*5
        gt_idx_bbox = np.concatenate((np.array(bboxes).astype(np.float32), np.array(cates)[:, None].astype(np.float32)), 1)
        if hico_pred.get(str(run_id)) is None:
            continue
        pred_idx_bbox = np.array(hico_pred[str(run_id)]).astype(np.float32)
        pred_idx_score = pred_idx_bbox[:,4]
        pred_idx_bbox = np.concatenate((pred_idx_bbox[:,:4], pred_idx_bbox[:, -1][:, None]), 1)
        pred_idx_score_list = np.argsort(-pred_idx_score) ## sort from large to samll
        
        num_gt, num_pred = gt_idx_bbox.shape[0], pred_idx_bbox.shape[0]
        gt_cate_map = gt_idx_bbox[:,-1][:,None].repeat(num_pred, 1)
        pred_cate_map = pred_idx_bbox[:,-1][None, :].repeat(num_gt, 0)
        cate_matching = (pred_cate_map==gt_cate_map).astype(np.float32) ## N1*N2
        obj_overlap = bbox_overlaps(gt_idx_bbox[:,:4], pred_idx_bbox[:,:4]) ## N1*N2

        for score_id in pred_idx_score_list.tolist():
            per_obj_overlap = obj_overlap[:, score_id]
            per_obj_overlap_binary = (per_obj_overlap>0.5).astype(np.float32)
            if per_obj_overlap_binary.sum()>0:
#                 print(per_obj_overlap.shape)
#                 print(cate_matching)
                matching_gt_id = np.argsort(-per_obj_overlap)[0]
                if cate_matching[matching_gt_id, score_id]==1:
                    if gt_cate_map[matching_gt_id, score_id] == 1: ## here to judge if human box
                        recall_person += 1
                    else:
                        recall_object += 1
                obj_overlap[matching_gt_id, :] = 0
        
        total_person += np.sum(gt_idx_bbox[:, 4]==1)
        total_object += (num_gt-np.sum(gt_idx_bbox[:, 4]==1))
        
        
    
print('total person:', total_person, 'recall person:', recall_person, 'recall:', recall_person/total_person)
print('total object:', total_object, 'recall object:', recall_object, 'recall:', recall_object/total_object)

37633
37631
total person: 76763 recall person: 57486 recall: 0.7488764118129828
total object: 61614 recall object: 35958 recall: 0.5836011296133996


In [23]:
total_person = 0
recall_person = 0

total_object = 0
recall_object = 0
print(len(hico_gt_test))
print(len(list(hico_pred_test.keys())))
for iddd, gt_info in enumerate(hico_gt_test):
#     print(iddd)
    img_id = gt_info['run_id']

    bboxes = gt_info['bboxes']
    cates = [0] * len(bboxes)
#     if img_id == 6919 or img_id ==9546:
#         continue
    if len(gt_info['annotations'])!=0:
        for anno in gt_info['annotations']:
            cates[anno['obj_bbox_idx']] = anno['obj_cate_id']
            cates[anno['hum_bbox_idx']] = 1
    
        ## N1*5, N2*5
        gt_idx_bbox = np.concatenate((np.array(bboxes).astype(np.float32), np.array(cates)[:, None].astype(np.float32)), 1)
        pred_idx_bbox = np.array(hico_pred_test[str(img_id)]).astype(np.float32)
        pred_idx_score = pred_idx_bbox[:,4]
        pred_idx_bbox = np.concatenate((pred_idx_bbox[:,:4], pred_idx_bbox[:, -1][:, None]), 1)
        pred_idx_score_list = np.argsort(-pred_idx_score) ## sort from large to samll
        
        num_gt, num_pred = gt_idx_bbox.shape[0], pred_idx_bbox.shape[0]
        gt_cate_map = gt_idx_bbox[:,-1][:,None].repeat(num_pred, 1)
        pred_cate_map = pred_idx_bbox[:,-1][None, :].repeat(num_gt, 0)
        cate_matching = (pred_cate_map==gt_cate_map).astype(np.float32) ## N1*N2
        obj_overlap = bbox_overlaps(gt_idx_bbox[:,:4], pred_idx_bbox[:,:4]) ## N1*N2

        for score_id in pred_idx_score_list.tolist():
            per_obj_overlap = obj_overlap[:, score_id]
            per_obj_overlap_binary = (per_obj_overlap>0.5).astype(np.float32)
            if per_obj_overlap_binary.sum()>0:
#                 print(per_obj_overlap.shape)
#                 print(cate_matching)
                matching_gt_id = np.argsort(-per_obj_overlap)[0]
                if cate_matching[matching_gt_id, score_id]==1:
                    if gt_cate_map[matching_gt_id, score_id] == 1: ## here to judge if human box
                        recall_person += 1
                    else:
                        recall_object += 1
                obj_overlap[matching_gt_id, :] = 0
        
        total_person += np.sum(gt_idx_bbox[:, 4]==1)
        total_object += (num_gt-np.sum(gt_idx_bbox[:, 4]==1))
        
        
    
print('total person:', total_person, 'recall person:', recall_person, 'recall:', recall_person/total_person)
print('total object:', total_object, 'recall object:', recall_object, 'recall:', recall_object/total_object)

9658
9545


KeyError: '6919'